In [813]:
# Run this cell first
from datascience import * 
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
#Data Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

plots.style.use('fivethirtyeight')

 Load the data. If there are labels other than p, n, or o, remove those rows. 

In [814]:
# write yourdata = data.where('sentiment',are.not_contained_in('?')).where('sentiment',are.not_equal_to('nan'))
                  
#checking just contains p n o 

data =Table.read_table('sentiment.csv')

#data.group('sentiment')
data


yorum,sentiment
ben bu filmi seyrettigimde çok etkilenmiştim.,p
Acaba Kate Winslet yerine Drew Barrymore daha mı başa ...,o
Bir kere Leonardo DiCaprio mükemmel oynamış.,p
val kilmer'ı bir çpk insanın sevme nedeni bu filmdir,p
11 oskar ödülüne gelince akademi hiç mi yanlış karar ...,o
'' Aşk filmi''.,o
Bu adamın 2 filmi var ( Kurtlarla Dansı izlemedim) 1.R ...,o
Kahinin sözlerine dikkat edin.,o
Sinemanın zevkine varıyorsunuz izlerken.,p
:) İlk izlediğimde çok etkilenmiştim.,p


In [815]:
data = data.where('sentiment',are.not_contained_in('?')).where('sentiment',are.not_equal_to('nan'))
                  
#checking just contains p n o 
data.group('sentiment')

sentiment,count
n,1
n,77
o,316
o,4
p,571
p,6


In [816]:
#the count column shows the just p n or o but there is a problem it would be caused by entering one space more
sentiment_column=[]

for i in data.column('sentiment'):
    new_str=i.replace(" ", "")
    sentiment_column.append(new_str)



data_corrected_sentiment=data.with_column('Corrected Sentiment',sentiment_column)
data_corrected_sentiment.group('Corrected Sentiment')

#As we can see the result show us it is corrected


Corrected Sentiment,count
n,78
o,320
p,577


In [817]:
data_c_sentiment=data_corrected_sentiment.move_column('Corrected Sentiment',1).drop('sentiment')
data_c_sentiment=data_c_sentiment.relabel('Corrected Sentiment','sentiment')
data_c_sentiment

yorum,sentiment
ben bu filmi seyrettigimde çok etkilenmiştim.,p
Acaba Kate Winslet yerine Drew Barrymore daha mı başa ...,o
Bir kere Leonardo DiCaprio mükemmel oynamış.,p
val kilmer'ı bir çpk insanın sevme nedeni bu filmdir,p
11 oskar ödülüne gelince akademi hiç mi yanlış karar ...,o
'' Aşk filmi''.,o
Bu adamın 2 filmi var ( Kurtlarla Dansı izlemedim) 1.R ...,o
Kahinin sözlerine dikkat edin.,o
Sinemanın zevkine varıyorsunuz izlerken.,p
:) İlk izlediğimde çok etkilenmiştim.,p


In [818]:
numeric_scores = Table().with_columns('sentiment',make_array('p','o','n'),'label',make_array(1,0,-1) )
numeric_scores
data2 = data_c_sentiment.join('sentiment',numeric_scores).move_column('sentiment',1)
data2.group('sentiment')


sentiment,count
n,78
o,320
p,577


Building a model which predicts the sentiment of each review randomly. Calculate its average accuracy using 5-fold cross validation.  We want to apply 5-fold cross validation without shuffling the data. That is, the first fold should be the first 20% of the data. The second fold should be the second 20% of the data.

In [309]:
#Just testing randint
predction=np.random.randint(-1,2, size=(100))


array([ 1, -1, -1,  1,  1, -1,  1,  0, -1, -1,  0,  1,  0,  0,  1,  1,  1,
        0, -1, -1,  0, -1,  1,  0,  0,  1,  1,  1, -1,  1,  1, -1,  0,  1,
       -1, -1,  1,  1, -1, -1, -1,  1,  0,  1, -1, -1, -1,  0, -1,  0,  0,
        0,  1,  0,  0,  0,  0,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1,
        0,  1,  1,  0, -1,  0,  1,  1,  0,  0,  0,  1, -1, -1,  1,  1, -1,
        1,  1,  1, -1, -1,  1,  1, -1,  1, -1,  0,  1, -1, -1, -1])

In [822]:

def prediction(train,test,k1):
    print()


def kfold(data,k1):
    
    
    def accuracy(prediction,test):
        numeric_scores = Table().with_columns('sentiment',make_array('p','o','n'),'label',make_array(1,0,-1) )
        
        test = test.join('sentiment',numeric_scores)
        
        result=[]
        for i in range(0,len(test.column(2))):
            if prediction[i]==test.column(2)[i]:
                result.append(1)
            else:
                result.append(0)
        result =np.array(result)
        
        return np.count_nonzero(result)/len(result)    
    
    index = np.arange(data.num_rows)
    #shuffled_index = index.sample(with_replacement=False)
    test_size = int(data.num_rows/k1)
    total_accuracy = 0
    
    for i in np.arange(k1):
        prediction=np.random.randint(-1,2, size=(test_size))
        test_index = index[i*test_size:(i+1)*test_size]
        train_index = np.setdiff1d(index, test_index)
        test = data.take(test_index)
        train = data.take(train_index)
        
        
        accr = accuracy(prediction,test) 
       
        total_accuracy=accr+total_accuracy
        print(str(i+1)+'th Fold result'+str(accr))
    return total_accuracy /k1


        
    
kfold(data2,5) 
    
    
    
    

1th Fold result0.2512820512820513
2th Fold result0.26666666666666666
3th Fold result0.31794871794871793
4th Fold result0.28717948717948716
5th Fold result0.3487179487179487


0.2943589743589744


 Building a model that predicts the sentiment of the reviews. You should define at least 10 features. Try to find the best machine learning model. Try to find the best feature set for this model. In the same 5-fold setting, if your model's average accuracy is 0.15 more than random model's performance, you will get full score from this question (still you should define 10 features.) *Show all your work for feature selection etc. including the models that could not perform well

***Data preprocess**

While preprocess I defined some stop word for Turkish Language, because they have no effective meaning in sentence like English word The, with etc.

In [641]:
emoticons = [('EMOTSMILEY', [':-)', ':)', '(:', '(-:', ] ) , ('EMOTLAUGH', [':-D', ':D', 'X-D', 'XD', 'xD', ] ), ('EMOTLOVE', ['<3', ':\*', ] ), ('EMOTWINK', [';-)', ';)', ';-D', ';D', '(;', '(-;', ] ), ('EMOTFROWN', [':-(', ':(', '(:', '(-:', ] ), ('EMOTCRY', [':,(', ':\'(', ':"(', ':(('] ),]

def escape_paren(arr):
    return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]

def regex_union(arr):
    return '(' + '|'.join( arr ) + ')'

emoticons_regex = [ (repl, re.compile(regex_union(escape_paren(regx))) ) for (repl, regx) in emoticons ]

def processEmoticons(text, subject='', query=[]):
    for (repl, regx) in emoticons_regex :
        text = re.sub(regx, ' '+repl+' ', text)
    return text

def processTweet(tweet):
    # process the tweets
    
    #Convert to lower case
    tweet = tweet.lower()
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

def getFeatureVector(tweet):
    stopWords =getStopWordList('C:\\Users\\ABRA\Desktop\\Dersler\\4.Sinif 2.Donem\\YAP101\\Hwbonus\\stopWords.txt')
  
    #split tweet into words
    str=''
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        
        str+=' '+w.lower()
    return str



def getWords(comment):
    stopWords =getStopWordList('C:\\Users\\ABRA\Desktop\\Dersler\\4.Sinif 2.Donem\\YAP101\\Hwbonus\\stopWords.txt')
  
    res=[]
    str=''
    words = comment.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        
        res.append(w.lower())
    return res

def getAllWordInList(List):
    res=[]
    for comment in List:
        for word in getWords(comment):
          
            res.append(word)
    
    return res
    

In [734]:
result=[]
for comment in data2.column('yorum'):
    result.append(getFeatureVector(comment))
data_with_preprocessed = data_c_sentiment.with_column('preprocessed comment',result)
data_with_preprocessed=data_with_preprocessed.move_column('preprocessed comment',0).drop('yorum')
data_with_preprocessed


preprocessed comment,sentiment
ama sinema tarihinin gelip titanik numara filmidir ders ...,p
benim filmindeki buradakinden iyidir,o
kaliteli basit anlam veremiyorum,p
izlenebilir,p
salondan off,o
jennifer lopez diyorum,o
titanik bana olmayan macera olmayan james cameron filmi ...,o
kotu,o
leonardo di caprio ergen,p
lopez olmasa,p


In [735]:
#As we can see some of row are null

data_with_preprocessed.group('preprocessed comment')

preprocessed comment,count
,106
abartmayi birakin,1
abd kamuoyunun ilgisini skandallardan spin doctoring i ...,1
ablam sagolsun kere yinede,1
acaba kate winslet drew barrymore olurdu,1
acyip benim,1
adaletisiz sosyal kapanda seleksiyonu,1
adam,1
adam kasabadan,1
adam sandler filmde oyunculukta,1


In [736]:
#remove blank row 
remove_indeks=[]

for i in range(0 ,len(data_with_preprocessed.column(0))):    
    if not data_with_preprocessed.column(0)[i].strip():
         remove_indeks.append(i)
    
data_with_preprocessed.remove(remove_indeks)
data_with_preprocessed.group('preprocessed comment')

preprocessed comment,count
abartmayi birakin,1
abd kamuoyunun ilgisini skandallardan spin doctoring i ...,1
ablam sagolsun kere yinede,1
acaba kate winslet drew barrymore olurdu,1
acyip benim,1
adaletisiz sosyal kapanda seleksiyonu,1
adam,1
adam kasabadan,1
adam sandler filmde oyunculukta,1
adam sandlerin damgasini culture club lezzet discoda ne ...,1


In [780]:
numeric_scores = Table().with_columns('sentiment',make_array('p','o','n'),'label',make_array(1,0,-1) )

data_with_preprocessed=data_with_preprocessed.drop('yorum')

size =data_with_preprocessed.num_rows
test=data_with_preprocessed.take(np.arange(0,int(size*0.2)))
train=data_with_preprocessed.take(np.arange(int(size*0.2),size))
train=train.join('sentiment',numeric_scores).move_column('sentiment',1)
test=test.join('sentiment',numeric_scores).move_column('sentiment',1)
train_label=train.column('label')
test_label=test.column('label')

  

**Feature Engineering**

In [776]:


negative_word_feature= make_array('kötü','berbat','başarısız','saçma','basit','sıkıldım','sıkıcı','abartı','gerek yok','kalitesiz','2','3','4',)
def negative_words_count(x,negative_list):
        
    count =0
    for word in negative_list:
        if word in x:
            count+=1
        
def negative_words_feature_row_based(row,negative_list):
    return negative_words_count(np.array(row)[0],negative_list)

negative_words_feature('hikaye basit olmasına rağmen bnce halen en iyi aşk filmidir','basit')


positive_word_feature= make_array('güzel','harika','süper','müthiş','akıcı','harika','iyi','kaliteli','muhteşem','helal','izlenmeli','tavsiye ','7','8','9','10')
def positive_words_count(x,positive_list):
        
    count =0
    for word in positive_list:
        if word in x:
            count+=1
        
def positive_words_count_row_based(row,positive_list):
    return positive_words_count(np.array(row)[0],positive_list)


positive_words_feature_words('hikaye basit olmasına rağmen bnce halen en iyi aşk filmidir','güzel')


def isNumberOfPositifWord(table):
    Feature_table =Table()
    for feature in positive_word_feature:
        result = []
        for i in range(0,table.num_rows):
            result.append(positive_words_feature_words(table.row(i)[0],feature))
        Feature_table=Feature_table.with_column('has '+str(feature),result)
    for feature in negative_word_feature:
        result = []
        for row in range(0,table.num_rows):
            result.append(negative_words_feature(table.row(i)[0],feature))
        Feature_table=Feature_table.with_column('has '+str(feature),result)
   

    return Feature_table
    



In [ ]:
#Converting table to future array
def convert_to_array_matrix(features):
    feature_vectors = []
    for i in np.arange(features.num_rows):
        row=[]
        for j in np.arange(features.num_columns):
            row.append( features.column(j).item(i))
        feature_vectors.append(row)
    return feature_vectors



In [783]:
def all_words_feature_words(x,word):
    if word in x:
        return 1
    else:
        return 0

    

In [784]:
#Whole words in training comments

train_word=getAllWordInList(train.column(0))


In [786]:
word_table=Table().with_column('Word',train_word)
word_table.group('Word').sort('count',descending=True)

def creating_feature_table(train_word,comment_list):
    Feature_table =Table()
    for feature in train_word:
        result = []
        for i in range(0,comment_list.num_rows):
            result.append(all_words_feature_words(comment_list.row(i)[0],feature))
        Feature_table=Feature_table.with_column('has '+str(feature),result)
    return Feature_table

#Train and Test Feature Vectors Table
Feature_train=creating_feature_table(train_word,train)
Feature_test=creating_feature_table(train_word,test)





In [787]:
Feature_test.num_columns

1200

#### I wonder which key word has a correlation between label and I remove the key word which has 0.4 smaller pearson with label because of my heuristic approch

In [790]:

labels=Feature_train.labels

def pearson_result(table,target):
    labels=table.labels
    labels=np.array(labels)
    pearson_of_feature = Table()
    res=[]
    for label in labels:   
        res.append(round(pearsonr(table.column(label),target)[0],2))
    return res


Eliminate Features Which Are not above r correlation 

In [793]:

def pearson_table(table,target,r):

    pearson_table=Table().with_columns('Label Name',labels,'Pearson with label',pearson_result(table,target))
    pearson_table=pearson_table.with_column('Abs Pearson with label',abs(pearson_table.column(1)))
    pearson_table=pearson_table.where('Abs Pearson with label',are.above(r)).sort('Abs Pearson with label',descending=True)
    return pearson_table

def eliminate_with_pearson(pearson_table,Feature_table):
    labels=pearson_table.column(0)
    labels=np.array(labels)
    pearson_of_feature = Table()
    Feature_table=Feature_table.select(labels)
    


eliminate_with_pearson(pearson_table(Feature_train,train_label,0.04),Feature_train)
eliminate_with_pearson(pearson_table(Feature_test,test_label,0.04),Feature_test)
Feature_train
#The below show us the all feature vector


has milleti | has kurtarmak | has kadarki | has tehlikeli | has zorundalar | has televizyonda | has herhangi | has pembe | has diziden | has olarak | has vasat | has amerikan | has yerimden | has kalkmadan | has soluksuz | has izledim | has izleyipte | has diyen | has yalan | has eminim | has sinema | has sandalye | has filmde | has anlayabilriz | has nin | has izleyin | has ahlak | has yerin | has dibine | has akademi | has zaman | has inanmak | has inanamyorm | has babam | has bundan | has cok | has yuksek | has filme | has puan | has verdim | has yorum | has yoruma | has zamanlar | has yere | has insan | has dejavuda | has kelimeyle | has var | has izlerken | has doluyor | has isimlerini | has zeki | has insana | has beynini | has beyni | has suni | has yaratarak | has unutturma | has beni | has james | has bond | has filminden | has flim | has tabiki | has billy | has zane | has vadisi | has kuzey | has irakta | has cool | has izlediim | has etkileyici | has filmlerden | has kate | has winsletin | has oynayan | has gloria | has stuartda | has leonardo | has di | has filmleri | has keinlikle | has izliyorum | has birisi | has varsa | has izlemeli | has izlemekten | has konusu | has seyircide | has izlenim | has kez | has seyrettim | has duygu | has safhaya | has geldi | has jean | has reno | has hasta | has hak | has eden | has felaket | has belli | has finali | has izlenmeye | has eski | has arada | has ondan | has puna | has diyecek | has duygudan | has mahrum | has bunun | has temposuda | has yorumlara | has leoda | has role | has hakediyo | has romantik | has duygusal | has filmdi | has kelimeykle | has titanik | has fb | has bana | has bide | has oyunuculuklarda | has olunca | has ortaya | has vede | has seyirlik | has saymazsak | has geldiinde | has zmnlar | has yetenek | has iyidir | has gelir | has izlemeyenlere | has tavsiye | has ediyorum | has herkesin | has izlemesi | has gereken | has yoktur | has heralde | has kalan | has winslet | has dev | has oyuncu | has kadrosu | has kurgusunun | has mementoyu | has tim | has hava | has etkisiz | has hale | has getirmek | has harekete | has duymaktan | has zevkler | has renkler | has yorumsuz | has zengin | has kalpli | has abd | has kamuoyunun | has ilgisini | has skandallardan | has spin | has doctoring | has i | has ince | has mizahi | has kraliyet | has iyileri | has listeye | has onu | has nedenini | has an | has adam | has sandlerin | has izlememezlik | has etmeyin | has kere | has rose | has olursa | has olsun | has bilimkugu | has idi | has dicaprio | has sevenlerini | has siteme | has bekliyorum | has filimin | has u | has oluyor | has izleyeli | has adaletisiz | has sosyal | has kapanda | has seleksiyonu | has adeta | has yapan | has holywood | has un | has zamanda | has toplumun | has beyaz | has perdeye | has aksiyon | has yonunden | has oynadigi | has diger | has filmlerine | has parca | has fark | has atiyor | has yer | has amerika | has iyisi | has annemle | has mutlu | has haber | has yorumumu | has nerden | has bilmiyorum | has sonucun | has dusunuyorum | has basit | has konu | has olabilir | has izlemeden | has demekki | has bende | has sulara | has kasabadan | has tasvir | has resmetme | has gayret | has eser | has etmeye | has geliyor | has diyenler | has yorumla | has anlatmak | has bunu | has belirtmekte | has fayda | has gazeteci | has ortalama | has filmden | has kes | has duruyor | has gelirse | has kalmaz | has sempati | has duymamakla | has sevimli | has senesinde | has titanic | has oyunculara | has verilseydi | has iileri | has tarzda | has finalinde | has oscar | has heyecan | has efektlerinde | has itici | has tedirgin | has kadro | has gusel | has leo | has bosmus | has sadce | has oldugu | has hata | has olmus | has orjinal | has cd | has sini | has neler | has verimez | has kevin | has costner | has kendisini | has insanlara | has mesaj | has korku | has kategorisinde | has istemiyorum | has val | has sevme | has nedeni | has filmdir | ha

K-Fold is not behaviour just K-fold it also contains Machine Learning Steps Except Datapreprocess because I have already done that

In [800]:
from sklearn import metrics
def kfold_with_model_name(data,k1,name,model):
    
    index = np.arange(data.num_rows)
    #shuffled_index = index.sample(with_replacement=False)
    test_size = int(data.num_rows/k1)
    total_accuracy = 0
    
    numeric_scores = Table().with_columns('sentiment',make_array('p','o','n'),'label',make_array(1,0,-1) )

    print(' K-Fold Start For This Model'+str(name))
    for i in np.arange(k1):
        
    
        test_index = index[i*test_size:(i+1)*test_size]
        train_index = np.setdiff1d(index, test_index)
        
        
        test = data.take(test_index)
        train = data.take(train_index)
        
        train=train.join('sentiment',numeric_scores).move_column('sentiment',1)
        test=test.join('sentiment',numeric_scores).move_column('sentiment',1)
        
        train_label=train.column('label')
        test_label=test.column('label')
        
        Feature_train=creating_feature_table(train_word,train)
        Feature_test=creating_feature_table(train_word,test)
        
        #Eliminate Feature which are below 0.04 value by heuristic approach
        
        eliminate_with_pearson(pearson_table(Feature_train,train_label,0.04),Feature_train)
        eliminate_with_pearson(pearson_table(Feature_test,test_label,0.04),Feature_test)
        
        model = model.fit(convert_to_array_matrix(Feature_train), train_label)
        y_pred = model.predict(convert_to_array_matrix(Feature_test))
        
        
        accr = accuracy_score(y_pred,test_label) 
       
        total_accuracy=accr+total_accuracy
        print(str(i+1)+'th Fold result'+str(accr))
    print()
    return total_accuracy /k1



**After my feature engineering steps I wonder to best model to fit my data there fore I writed belowed code, and important thing in here I used many features to find minimum error therefore it takes 2 3 hour to reach result ***

In [802]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

  


models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Naive Bayes', GaussianNB()))
models.append(('Decision Tree (CART)',DecisionTreeClassifier())) 
models.append(('K-NN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('BaggingClassifier', BaggingClassifier()))
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('MLPClassifier', MLPClassifier()))


for name, model in models:
    
    print(str(name)+"model k-fold result"+str(kfold_with_model_name(data_with_preprocessed,5,name,model)))
    
    

 K-Fold Start For This ModelLogistic Regression
1th Fold result0.43352601156069365
2th Fold result0.45664739884393063
3th Fold result0.5317919075144508
4th Fold result0.5838150289017341
5th Fold result0.5780346820809249

Logistic Regressionmodel k-fold result0.5167630057803467
 K-Fold Start For This ModelNaive Bayes
1th Fold result0.26011560693641617
2th Fold result0.4277456647398844
3th Fold result0.3063583815028902
4th Fold result0.3468208092485549
5th Fold result0.3815028901734104

Naive Bayesmodel k-fold result0.3445086705202312
 K-Fold Start For This ModelDecision Tree (CART)
1th Fold result0.4277456647398844
2th Fold result0.45664739884393063
3th Fold result0.4682080924855491
4th Fold result0.5375722543352601
5th Fold result0.5028901734104047

Decision Tree (CART)model k-fold result0.47861271676300576
 K-Fold Start For This ModelK-NN
1th Fold result0.43352601156069365
2th Fold result0.47398843930635837
3th Fold result0.4624277456647399
4th Fold result0.4682080924855491
5th Fold r

after kfold_with_model_name 3 hours run we conclude SVM model has best fit with result **0.5907514450867051** I guess this is enough for showing its better than random model

After finding the 0.15 better performance model then random model I wonder is there any other feature combination to fit my model and used some of feature extraction method and library

Using Count Vectorizer

In [823]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(train.column(0))
X = cv.transform(train.column(0))
X_test = cv.transform(test.column(0))


In [824]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier


models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Naive Bayes', GaussianNB()))
models.append(('Decision Tree (CART)',DecisionTreeClassifier())) 
models.append(('K-NN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('BaggingClassifier', BaggingClassifier()))
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('MLPClassifier', MLPClassifier()))


for name, model in models:
    model = model.fit(X.toarray(), train.column(2))
    y_pred = model.predict(X_test.toarray())
    from sklearn import metrics

    print("%s -> ACC: %%%.2f" % (name,metrics.accuracy_score(test.column(2), y_pred)*100))

Logistic Regression -> ACC: %42.77
Naive Bayes -> ACC: %28.32
Decision Tree (CART) -> ACC: %38.15
K-NN -> ACC: %42.20
SVM -> ACC: %45.09
Gradient Boosting Classifier -> ACC: %42.77
AdaBoostClassifier -> ACC: %41.62
BaggingClassifier -> ACC: %37.57
RandomForestClassifier -> ACC: %39.88
MLPClassifier -> ACC: %36.42


Using TfidfVectorizer

In [811]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

vect = TfidfVectorizer(max_df=0.9, min_df=3, sublinear_tf=True)

vect.fit(train.column(0))
X = vect.transform(train.column(0))
X_test = vect.transform(test.column(0))



In [812]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier


models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Naive Bayes', GaussianNB()))
models.append(('Decision Tree (CART)',DecisionTreeClassifier())) 
models.append(('K-NN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('Gradient Boosting Classifier', GradientBoostingClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('BaggingClassifier', BaggingClassifier()))
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('MLPClassifier', MLPClassifier()))


for name, model in models:
    model = model.fit(X.toarray(), train.column(2))
    y_pred = model.predict(X_test.toarray())
    from sklearn import metrics

    print("%s -> ACC: %%%.2f" % (name,metrics.accuracy_score(test.column(2), y_pred)*100))

Logistic Regression -> ACC: %43.93
Naive Bayes -> ACC: %18.50
Decision Tree (CART) -> ACC: %37.57
K-NN -> ACC: %25.43
SVM -> ACC: %45.09
Gradient Boosting Classifier -> ACC: %42.20
AdaBoostClassifier -> ACC: %39.31
BaggingClassifier -> ACC: %39.88
RandomForestClassifier -> ACC: %39.88
MLPClassifier -> ACC: %42.20
